In [56]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.feature_selection import mutual_info_classif
import pydotplus
from IPython.display import Image
from pandas.plotting import scatter_matrix
from math import log, e

In [57]:
def retrieve_all(Ts): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [58]:
#riceve la lista di coppie dei motifs per ogni record(Ts), e resittuisce lista di valori singoli

def motifsFilter(MotifsList): 
    l2=np.array([])
    for i in range (len(MotifsList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(MotifsList['Motif'].loc[i])
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=MotifsList['Motif'].loc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista 
    
        MotifsList['Motif'].loc[i]=l2
        l2=np.array([]) #svuoto array
    
    return MotifsList
    

In [59]:
#carico dataset in dataFrame
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
df = pd.DataFrame(dataset[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
#trasformo da stringa a numero il campo target 
le = LabelEncoder()
num_classes = le.fit_transform(df['target'])
df['target']=num_classes
df['TsIndex']=np.arange(len(df))

print(df)

        att1      att2      att3      att4      att5      att6      att7  \
0  -0.464276 -0.555048 -0.842843 -0.865895 -0.936396 -0.817270 -0.263612   
1  -0.896972 -0.685686 -1.351382 -1.458667 -1.165346 -1.403929 -1.821800   
2  -0.464696 -0.567739 -0.032023 -0.635046 -0.602826 -0.266856 -0.267061   
3  -0.187190 -0.620808 -0.815661 -0.521398 -0.790423 -0.967517 -1.487006   
4  -1.136017 -1.319195 -1.844624 -0.788692 -0.251715 -1.487603 -0.668764   
5  -0.568709 -0.888118 -0.047977 -0.038036 -0.483068 -1.094689 -0.953453   
6  -0.965839 -1.644583 -1.805935 -1.499658 -0.920088 -1.115706 -0.650195   
7  -0.264302 -0.290925 -0.598567 -0.518863 -0.488594 -0.407091 -1.071489   
8  -0.803716 -0.048161 -1.299278 -1.072280 -1.004018 -0.511452 -1.173215   
9  -1.186422 -0.893197 -0.977737 -0.634109 -1.384820 -1.271926 -1.212171   
10 -0.653308 -1.295185 -0.457384 -1.092036 -1.682346 -0.461811 -0.099362   
11 -0.541876 -1.016107 -1.079902 -1.185665 -0.792452 -0.754593 -0.057041   
12 -1.945327

In [60]:
window_size=5
#diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
diz={'Motif':[]}

for i in range(30):
    Ts = np.array(df.iloc[i].values)
    mp,mot,motif_dist,dis = retrieve_all(Ts[0:128])
    diz['Motif'].insert(i, mot)


MotifsList = pd.DataFrame(diz)
print('Motif estratti')
print(MotifsList)

Motif estratti
                           Motif
0            [[9, 93], [26, 85]]
1        [[105, 108], [54, 113]]
2           [[17, 56], [40, 94]]
3     [[48, 61], [12, 109, 115]]
4                     [[27, 95]]
5      [[45, 60], [13, 17, 122]]
6            [[7, 48], [37, 43]]
7            [[2, 92], [24, 56]]
8            [[1, 11], [29, 52]]
9           [[50, 103], [0, 33]]
10          [[11, 26], [17, 75]]
11           [[31, 55], [7, 60]]
12                   [[53, 105]]
13          [[79, 84], [6, 106]]
14                    [[59, 70]]
15           [[2, 91], [45, 59]]
16  [[28, 44, 86, 117], [3, 34]]
17         [[14, 20], [35, 117]]
18           [[13, 71], [3, 48]]
19           [[19, 55], [6, 67]]
20          [[19, 78], [39, 85]]
21                    [[26, 84]]
22          [[18, 28], [22, 65]]
23         [[27, 116], [42, 63]]
24       [[8, 74, 87], [28, 65]]
25                    [[16, 81]]
26          [[67, 78], [59, 86]]
27         [[16, 23], [39, 111]]
28          [[11, 92], [41, 

In [61]:
MotifsList=motifsFilter(MotifsList)
print(MotifsList['Motif'])
Ts = np.array(df.iloc[0].values)
print(Ts[9:(9+5)])
print(Ts[26:(26+5)])

0       [9.0, 26.0]
1     [105.0, 54.0]
2      [17.0, 40.0]
3      [48.0, 12.0]
4            [27.0]
5      [45.0, 13.0]
6       [7.0, 37.0]
7       [2.0, 24.0]
8       [1.0, 29.0]
9       [50.0, 0.0]
10     [11.0, 17.0]
11      [31.0, 7.0]
12           [53.0]
13      [79.0, 6.0]
14           [59.0]
15      [2.0, 45.0]
16      [28.0, 3.0]
17     [14.0, 35.0]
18      [13.0, 3.0]
19      [19.0, 6.0]
20     [19.0, 39.0]
21           [26.0]
22     [18.0, 22.0]
23     [27.0, 42.0]
24      [8.0, 28.0]
25           [16.0]
26     [67.0, 59.0]
27     [16.0, 39.0]
28     [11.0, 41.0]
29      [6.0, 15.0]
Name: Motif, dtype: object
[-0.91830825 -0.92210226 -0.98448828 -1.2880511  -1.14346   ]
[-1.2977275  1.3314643  0.9357379  1.2360468  1.0560677]


In [62]:
#Fun MOTIFS Returns tuple (motifs, distances)
    #motifs: a list of lists of indexes representing the motif starting locations.
    #distances: list of minimum distances for each motif
#riceve la lista degli indici di partenza dei motif, e le Ts, per ognuno di essi(motifs) genera un DistanceProfile con ogni Ts
#e prende il valore(distanza) minore con tale Ts

def retrieveSubSequenceDist(df,MotifsList):
    diz={'Index':[],'TsIndex':[],'motifStartingIndex':[],'Used':[],'SubSequenceDistArray':[]}
    
    
    #dfSubSeqDist -> TsIndex= indice della Ts a cui appartiene il candidato shapelet
    
    #                motifStartingIndex= indice di posizione di partenza del candidato shapelet nella TS
    
    #                Used= se è gia stato usato o meno nel DTree
    
    #                SubSequenceDistArray= array 2D in cui ogni riga i ha 
    #                indice della Ts(dell'elenco nel df), la distanza minima tra il candidato e la Ts i-esima,la classe della Ts-iesima
    #                NB: SubSequenceDistArray viene usato nel DTREE, contenuti nei dataset che vengono splittati
    
    # NB: la coppia TsIndex e motifStartingIndex IDENTIFICANO UNIVOCAMENTE un candidato Shapelet
    # L'INDICE DELLA Ts SI RIFERISCE ALLA POSIZIONE CHE OCCUPA NELLA MOTIFS LIST               
    
    
    counter=0
    
    for i in range(len(MotifsList['Motif'])):
        numMotif=len(MotifsList['Motif'].loc[i]) #quanti starting index di motif ci sono in pos i
        for j in range (numMotif):
            l1=MotifsList['Motif'].loc[i] #lista di indice i in MotifsList
            startingIndex=l1[j] #indice di inizio del motif
            #print(startingIndex)
            TsContainingCandidateShapelet = np.array(df.iloc[i].values) #Ts contenente candidato shapelet
            minValueList = np.array([]) #array contenente le subSeqDist calcolate dalla data SubSeq con tutte le ts
            for k in range (len(df)):
                TsToCompare = np.array(df.iloc[k].values)
                class_value=TsToCompare[128]
                TsToCompare=TsToCompare[0:128]
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueList=np.append(minValueList,min(Dp[0])) #Dp[0] contiene il Dp effettivo
                
            diz['Index'].insert(counter, counter)
            diz['TsIndex'].insert(counter, i)
            diz['motifStartingIndex'].insert(counter, startingIndex)
            diz['Used'].insert(counter, False)
            
            dizForSubSeqArray=pd.DataFrame()
            dizForSubSeqArray['TsIndex']=np.arange(len(df))
            dizForSubSeqArray['minValueList']=minValueList
            dizForSubSeqArray['class']=num_classes
            
            diz['SubSequenceDistArray'].insert(counter, dizForSubSeqArray)

            
            counter=counter+1
    
    dfSubSeqDist=pd.DataFrame(diz)
    return dfSubSeqDist

In [63]:
dfSubSeqDist=pd.DataFrame()
dfSubSeqDist=retrieveSubSequenceDist(df,MotifsList)
dfSubSeqDist.set_index('Index')


print('')

#for a,b,c in zip(dfSubSeqDist['SubSequenceDistArray'].iloc[0][0], dfSubSeqDist['SubSequenceDistArray'].iloc[0][1],dfSubSeqDist['SubSequenceDistArray'].iloc[0][2]):
#    print(a,b,c)
    
print(dfSubSeqDist['SubSequenceDistArray'].iloc[0])
print(type(dfSubSeqDist['SubSequenceDistArray'].iloc[0]))

#FIN QUI: PER OGNI MOTIF ESTRATTO (IDENTIFICATO UNIVOCAMENTE DALLA TS DI APPARTENZENZA E INDICE DI PARTENZA IN ESSA)
#         HO CALCOLATO UN DPROFILE CON OGNI TS, PRESO IL VALORE(DISTANZA) MINORE DA OGNI DPROFILE 
#         E COSTRUITO UN VETTORE CONTENENTE QUINDI LA MINIMA DISTANZA TRA UN MOTIF FISSATO E TUTTE LE TS (SubSequenceDistArray)
#         IN TALE ARRAY, ACCANTO AD OGNI MINIMA DISTANZA, SALVO ANCHE L'INDICE E LA CLASSE DELLA TS CON CUI è STATA CALCOLATA
#         TALE INFO SONO FONDAMENTALI NEL DTREE


    TsIndex  minValueList  class
0         0      0.000000      0
1         1      0.983997      0
2         2      0.512876      0
3         3      0.476238      2
4         4      0.612164      1
5         5      0.522029      1
6         6      0.640046      0
7         7      0.852676      2
8         8      0.688860      1
9         9      0.820811      0
10       10      0.531972      1
11       11      0.702819      1
12       12      0.528394      0
13       13      0.492519      1
14       14      0.392114      1
15       15      0.874073      2
16       16      0.652351      1
17       17      0.516231      1
18       18      0.384127      0
19       19      0.661830      2
20       20      0.657738      1
21       21      0.261145      1
22       22      0.653865      0
23       23      0.410703      0
24       24      0.473249      2
25       25      0.967778      2
26       26      0.623220      1
27       27      0.385411      2
28       28      0.648161      2
29       

In [64]:
#SUBSEQDISARRAY: [INDICE TS][VALORE MINIMO CALCOLATO][CLASSE TS]

In [65]:
#dataset(dframe)= dfSubSeqDist['SubSequenceDistArray'].iloc[i] cioe la lista di distanze minime tra candidato i-esimo e tutte le TS      
def split(dataset,value): 
    dizLeft=pd.DataFrame(columns=['TsIndex','minValueList','class'])
    dizRight=pd.DataFrame(columns=['TsIndex','minValueList','class'])
    index=0
    for i in range(len(dataset)):
        if dataset.iloc[i]['minValueList'] < value:
            #dizLeft.[index]=dataset.iloc[i] #in ogni split aggiungo la distanza tra candidato e Ts, e la classe della Ts
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
        index+=1
    return dizLeft, dizRight



In [66]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset, num_classes_unique):
    numPattern=len(dataset)
    print('numPattern: '+str(numPattern))
    relativeFrequencies=list()
    counter=0 #num pattern di ogni classe nel dataset
    #ottengo frequenze relative
    for i in (num_classes_unique):
        for j in range(numPattern):
            if(dataset.iloc[j]['class']==i):
                counter+=1
        relativeFrequencies.append(counter/numPattern)
        counter=0
    #calcolo entropia dataset
    print('relativeFrequencies: '+str(relativeFrequencies))
    entropy=(relativeFrequencies[0]*log(relativeFrequencies[0],2))
    for k in (relativeFrequencies[1:]):
        if(k!=0):
            entropy+=(k*log(k,2))
    return -entropy
    
    

In [67]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli
def computeGain(DatasetParent,Dleft,Dright,num_classes_unique):
    entropyParent=computeEntropy(DatasetParent,num_classes_unique)
    entropyLeft=computeEntropy(Dleft,num_classes_unique)
    entropyRight=computeEntropy(Dright,num_classes_unique)
    gain=entropyParent
    summation=( ((len(Dleft)/len(DatasetParent))*entropyLeft) +  ((len(Dright)/len(DatasetParent))*entropyRight) )
    print('entropyParent: '+str(entropyParent))
    print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [68]:
#media=np.median((dfSubSeqDist['SubSequenceDistArray'].iloc[0]['minValueList']))
l,r=split(dfSubSeqDist['SubSequenceDistArray'].iloc[0],0.62)
#print(media)
print(l)
print('')
#print(r)
num_classes_unique=np.unique(num_classes)
#print(computeEntropy(dfSubSeqDist['SubSequenceDistArray'].iloc[0],num_classes_unique))

#print(computeEntropy(l,num_classes_unique))

#print(computeEntropy(r,num_classes_unique))
#gain=computeGain(dfSubSeqDist['SubSequenceDistArray'].iloc[0],l,r,num_classes_unique)
#print('GAIN: '+str(gain))


l1,r1=split(l,0.45)
print(l1)
print('')
print(r1)

#print(computeEntropy(l1,num_classes_unique))

#print(computeEntropy(r1,num_classes_unique))
gain=computeGain(l,l1,r1,num_classes_unique)
print('GAIN: '+str(gain))


    TsIndex  minValueList  class
0       0.0      0.000000    0.0
1       2.0      0.512876    0.0
2       3.0      0.476238    2.0
3       4.0      0.612164    1.0
4       5.0      0.522029    1.0
5      10.0      0.531972    1.0
6      12.0      0.528394    0.0
7      13.0      0.492519    1.0
8      14.0      0.392114    1.0
9      17.0      0.516231    1.0
10     18.0      0.384127    0.0
11     21.0      0.261145    1.0
12     23.0      0.410703    0.0
13     24.0      0.473249    2.0
14     27.0      0.385411    2.0

   TsIndex  minValueList  class
0      0.0      0.000000    0.0
1     14.0      0.392114    1.0
2     18.0      0.384127    0.0
3     21.0      0.261145    1.0
4     23.0      0.410703    0.0
5     27.0      0.385411    2.0

   TsIndex  minValueList  class
0      2.0      0.512876    0.0
1      3.0      0.476238    2.0
2      4.0      0.612164    1.0
3      5.0      0.522029    1.0
4     10.0      0.531972    1.0
5     12.0      0.528394    0.0
6     13.0      0.4925

In [55]:
print(mutual_info_classif(l['minValueList'].values, l['class'].values, discrete_features=False))
print(type(l['minValueList'].values))
#print(l[[:,-1]])

ValueError: Expected 2D array, got 1D array instead:
array=[0.         0.51287624 0.47623843 0.61216377 0.52202901 0.53197164
 0.52839434 0.49251928 0.39211419 0.51623058 0.38412704 0.26114486
 0.41070267 0.47324857 0.3854114 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.